In [ ]:
import pyocr
import importlib
import sys
import time
 
importlib.reload(sys)
time1 = time.time()
# print("初始时间为：",time1)
 
import os.path
from pdfminer.pdfparser import  PDFParser,PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LTTextBoxHorizontal,LAParams
from pdfminer.pdfinterp import PDFTextExtractionNotAllowed
 
text_path = r'words-words.pdf'
# text_path = r'photo-words.pdf'
 
def parse():
    '''解析PDF文本，并保存到TXT文件中'''
    fp = open(text_path,'rb')
    #用文件对象创建一个PDF文档分析器
    parser = PDFParser(fp)
    #创建一个PDF文档
    doc = PDFDocument()
    #连接分析器，与文档对象
    parser.set_document(doc)
    doc.set_parser(parser)
 
    #提供初始化密码，如果没有密码，就创建一个空的字符串
    doc.initialize()
 
    #检测文档是否提供txt转换，不提供就忽略
    if not doc.is_extractable:
        raise PDFTextExtractionNotAllowed
    else:
        #创建PDF，资源管理器，来共享资源
        rsrcmgr = PDFResourceManager()
        #创建一个PDF设备对象
        laparams = LAParams()
        device = PDFPageAggregator(rsrcmgr,laparams=laparams)
        #创建一个PDF解释其对象
        interpreter = PDFPageInterpreter(rsrcmgr,device)
 
        #循环遍历列表，每次处理一个page内容
        # doc.get_pages() 获取page列表
        for page in doc.get_pages():
            interpreter.process_page(page)
            #接受该页面的LTPage对象
            layout = device.get_result()
            # 这里layout是一个LTPage对象 里面存放着 这个page解析出的各种对象
            # 一般包括LTTextBox, LTFigure, LTImage, LTTextBoxHorizontal 等等
            # 想要获取文本就获得对象的text属性，
            for x in layout:
                if(isinstance(x,LTTextBoxHorizontal)):
                    with open(r'blog.txt','a') as f:
                        results = x.get_text()
                        print(results)
                        f.write(results  +"\n")
parse()

#提取关键词
def getKeywords(filename,filename_key):
    file = open(filename, 'r',encoding='utf-8')
    file_k = open(filename_key,'a',encoding='utf-8')
    content = file.read()
    startStr = 'Keywords:'
    endStr = '\n\n'
    try:
        startIndex = content.index(startStr)
        endIndex = content[startIndex:].index(endStr)
        con_key = content[startIndex+len(startStr):startIndex+endIndex]
        file.close()
        file_k.write(con_key)
        file_k.write('\n')
        file_k.close()
    except:
        print('该文件无关键字')
        file.close()
        file_k.close()
                        
#提取网页内容
import urllib.request
import re
import os
from bs4 import BeautifulSoup
# open the url and read
def getHtml(url):
    headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36'} 
    req = urllib.request.Request(url=url, headers=headers) 
    page = urllib.request.urlopen(req)
#     html = page.read().decode('gb2312','ignore')
    html = page.read().decode('gb2312')
    page.close()
    return html

raw_url = 'http://zkxb.xjtu.edu.cn/oa/scriptlsit.aspx?kind=Issue&issnum=%B5%DA12%C6%DA&year=2020%C4%EA'
html = getHtml(raw_url)
# print(html)
bs = BeautifulSoup(html, 'html.parser') #解析网页
hyperlink = bs.find_all('a')  #获取所有超链接
# print(hyperlink)
file = open('demo.txt', 'w')
 
for h in hyperlink:
    hh = h.get('href')
    if hh and 'Sid=' in hh and 'pdff' not in hh:  #筛选博客链接
        print(hh)
        file.write(hh)   #写入到“demo.txt”文件中
        file.write('\n')
        
file.close()

In [ ]:
#统计每个关键词出现的次数
import collections
import os
with open('blog_key.txt') as file1:#打开文本文件
str1=file1.read().split(' ')#将文章按照空格划分开
print "原文本:\n %s"% str1
print "\n各单词出现的次数：\n %s" % collections.Counter(str1)
print collections.Counter(str1)['a']#以字典的形式存储，每个字符对应的键值就是在文本中出现的次数

In [ ]:
#绘制词频云图
import matplotlib
import matplotlib.pyplot as plt #数据可视化
import jieba #词语切割
import wordcloud #分词
from wordcloud import WordCloud,ImageColorGenerator,STOPWORDS #词云，颜色生成器，停止
import numpy as np #科学计算
from PIL import Image #处理图片

def ciyun():
    #打开文本
    with open('blog_key.txt','r',encoding='UTF-8') as f:  # 打开新的文本转码为utf-8
        textfile= f.read()  #读取文本内容
    wordlist = jieba.cut_for_search(textfile)#切割词语
    space_list = ' '.join(wordlist) # 链接词语
    backgroud = np.array(Image.open('beijin.png')) #背景图片，只有黑白图才能按照形状生成词云
    mywordcloud = WordCloud(width=14000, height=2200,
                            background_color=None, mode="RGBA",#背景颜色
                            mask=backgroud, #写字用的背景图，从图片中提取颜色
                            max_words=500, #最大词语数
                            stopwords=STOPWORDS,#停止的默认词语
                            font_path='simkai.ttf',#源码自带字体
                            max_font_size=200,#最大字体尺寸
                            random_state=50,#随机角度
                            scale=1).generate(space_list) #生成词云
    image_color = ImageColorGenerator(backgroud)#生成词云的颜色
    plt.imshow(mywordcloud) #显示词云
    plt.axis('off') #关闭坐标（x,y轴）
    plt.savefig('cytu.png') #保存图片
    plt.show()#显示

def main():
    ciyun()
    
main()